- choropleth map for all the teams and their favorite (worst) ground
- best openers and their help in win
- best finishers and their help in win
- bowlers and batsman pair to watch
- map of most destructive batsman

In [1]:
import pandas as pd
from collections import Counter

In [2]:
df = pd.read_csv('./datasets/matches.dmp')

In [3]:
df.head()

,id,season,city,date,team1,team2,toss_winner,toss_decision,result,dl_applied,winner,win_by_runs,win_by_wickets,player_of_match,venue,umpire1,umpire2,umpire3
0,1,2008,Bangalore,2008-04-18,Kolkata Knight Riders,Royal Challengers Bangalore,Royal Challengers Bangalore,field,normal,0,Kolkata Knight Riders,140,0,BB McCullum,M Chinnaswamy Stadium,Asad Rauf,RE Koertzen,NaN
1,2,2008,Chandigarh,2008-04-19,Chennai Super Kings,Kings XI Punjab,Chennai Super Kings,bat,normal,0,Chennai Super Kings,33,0,MEK Hussey,"Punjab Cricket Association Stadium, Mohali",MR Benson,SL Shastri,NaN
2,3,2008,Delhi,2008-04-19,Rajasthan Royals,Delhi Daredevils,Rajasthan Royals,bat,normal,0,Delhi Daredevils,0,9,MF Maharoof,Feroz Shah Kotla,Aleem Dar,GA Pratapkumar,NaN
3,4,2008,Mumbai,2008-04-20,Mumbai Indians,Royal Challengers Bangalore,Mumbai Indians,bat,normal,0,Royal Challengers Bangalore,0,5,MV Boucher,Wankhede Stadium,SJ Davis,DJ Harper,NaN
4,5,2008,Kolkata,2008-04-20,Deccan Chargers,Kolkata Knight Riders,Deccan Chargers,bat,normal,0,Kolkata Knight Riders,0,5,DJ Hussey,Eden Gardens,BF Bowden,K Hariharan,NaN


### Teams

In [4]:
teams = list(df['team1'].unique())
teams

['Kolkata Knight Riders',
 'Chennai Super Kings',
 'Rajasthan Royals',
 'Mumbai Indians',
 'Deccan Chargers',
 'Kings XI Punjab',
 'Royal Challengers Bangalore',
 'Delhi Daredevils',
 'Kochi Tuskers Kerala',
 'Pune Warriors',
 'Sunrisers Hyderabad',
 'Rising Pune Supergiants',
 'Gujarat Lions']

### Grounds

In [5]:
grounds = list(df['venue'].unique())
grounds

['M Chinnaswamy Stadium',
 'Punjab Cricket Association Stadium, Mohali',
 'Feroz Shah Kotla',
 'Wankhede Stadium',
 'Eden Gardens',
 'Sawai Mansingh Stadium',
 'Rajiv Gandhi International Stadium, Uppal',
 'MA Chidambaram Stadium, Chepauk',
 'Dr DY Patil Sports Academy',
 'Newlands',
 "St George's Park",
 'Kingsmead',
 'SuperSport Park',
 'Buffalo Park',
 'New Wanderers Stadium',
 'De Beers Diamond Oval',
 'OUTsurance Oval',
 'Brabourne Stadium',
 'Sardar Patel Stadium, Motera',
 'Barabati Stadium',
 'Vidarbha Cricket Association Stadium, Jamtha',
 'Himachal Pradesh Cricket Association Stadium',
 'Nehru Stadium',
 'Holkar Cricket Stadium',
 'Dr. Y.S. Rajasekhara Reddy ACA-VDCA Cricket Stadium',
 'Subrata Roy Sahara Stadium',
 'Shaheed Veer Narayan Singh International Stadium',
 'JSCA International Stadium Complex',
 'Sheikh Zayed Stadium',
 'Sharjah Cricket Stadium',
 'Dubai International Cricket Stadium',
 'Maharashtra Cricket Association Stadium',
 'Punjab Cricket Association IS Bind

In [45]:
non_indian_grounds = [
     'Newlands',
     "St George's Park",
     'Kingsmead',
     'SuperSport Park',
     'Buffalo Park',
     'New Wanderers Stadium',
     'De Beers Diamond Oval',
     'OUTsurance Oval',
     'Sheikh Zayed Stadium',
     'Sharjah Cricket Stadium',
     'Dubai International Cricket Stadium',
]

def is_indian_ground(ground_name):
    return False if ground_name in non_indian_grounds else True

In [50]:
def get_grounds_results(team_name):
    # append the team1/team2 appearance
    df1 = df[df['team1'] == team_name]
    df2 = df[df['team2'] == team_name]
    _df = df1.append(df2)
    
    # win and lose for each ground
    _won = Counter(filter(lambda g: is_indian_ground(g), list(_df[_df['winner'] == team_name]['venue'])))
    _lost = Counter(filter(lambda g: is_indian_ground(g), list(_df[_df['winner'] != team_name]['venue'])))
    
    # total grounds the team has played on
    _grounds = set(_won).union(set(_lost))
    
    _matches = []
    
    # Mohali ground has multiple names in dataset
    mohali_v1_name = "Punjab Cricket Association Stadium, Mohali"
    mohali_v2_name = "Punjab Cricket Association IS Bindra Stadium, Mohali"
    mohali_v2_won, mohali_v2_lost = _won.get(mohali_v2_name, 0), _lost.get(mohali_v2_name, 0)
    
    # iterate over all grounds
    for g in list(_grounds):
        # ignore the mohali grounds
        if g not in [mohali_v1_name, mohali_v2_name]:
            _matches.append({
                'ground': g,
                'won': _won.get(g, 0),
                'lost': _lost.get(g, 0),
            })  
            
    # unify the mohali results
    _matches.append({
        'ground': mohali_v1_name,
        'won': _won.get(mohali_v1_name, 0) + (mohali_v2_won),
        'lost': _lost.get(mohali_v1_name, 0) + (mohali_v2_lost),
    })
    
    result = {
        team_name: _matches
    }
    
    return result

In [51]:
for t in teams:
    print('--------------------\n', t)
    print(get_grounds_results(t))

--------------------
 Kolkata Knight Riders
{'Kolkata Knight Riders': [{'ground': 'M Chinnaswamy Stadium', 'won': 4, 'lost': 4}, {'ground': 'Barabati Stadium', 'won': 3, 'lost': 0}, {'ground': 'Eden Gardens', 'won': 33, 'lost': 18}, {'ground': 'JSCA International Stadium Complex', 'won': 1, 'lost': 2}, {'ground': 'Sardar Patel Stadium, Motera', 'won': 0, 'lost': 2}, {'ground': 'Wankhede Stadium', 'won': 1, 'lost': 5}, {'ground': 'Rajiv Gandhi International Stadium, Uppal', 'won': 4, 'lost': 1}, {'ground': 'Maharashtra Cricket Association Stadium', 'won': 2, 'lost': 0}, {'ground': 'Dr. Y.S. Rajasekhara Reddy ACA-VDCA Cricket Stadium', 'won': 0, 'lost': 1}, {'ground': 'Brabourne Stadium', 'won': 0, 'lost': 2}, {'ground': 'Green Park', 'won': 0, 'lost': 1}, {'ground': 'Shaheed Veer Narayan Singh International Stadium', 'won': 0, 'lost': 1}, {'ground': 'Nehru Stadium', 'won': 0, 'lost': 1}, {'ground': 'Sawai Mansingh Stadium', 'won': 1, 'lost': 3}, {'ground': 'Dr DY Patil Sports Academy', 

In [49]:
df1 = df[df['team1'] == 'Gujarat Lions']
df2 = df[df['team2'] == 'Gujarat Lions']
df1.append(df2).iloc[6]['venue']

'Punjab Cricket Association IS Bindra Stadium, Mohali'